## Convolutional Neural Networks

In this assignment, we will learn about convolutional neural networks. We will create a CNN and learn to classify image data.

In this lecture, we will use the image data generator to classify our data. The data is loaded below:

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization 
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model

In [3]:
train_data_dir = '/content/drive/MyDrive/dogs-vs-cats-processed/train/'
validation_data_dir = '/content/drive/MyDrive/dogs-vs-cats-processed/test/'

img_width, img_height = 150, 150
batch_size = 80

In [4]:
#This block of code is used to ensure the input shape is correct

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

Define a train data generator with shear range of 0.3, zoom range of 0.1 and rescale to 1./255 (note that we must make 1 a float to produce a correct fraction). Use the ImageDataGenerator function.

In [5]:
# Answer below:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.3,
    zoom_range=0.1,
    horizontal_flip=True
)


Define a test data generator that only rescales to 1./255. Use the ImageDataGenerator function.

In [6]:
# Answer below:
test_datagen = ImageDataGenerator(rescale=1. /255) 
#only need to rescale for test

The train generator and the test generator are defined below:

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    #shuffle=False,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


We'll start with a simple model. In CNNs, we first convolve the to extract features and then we add the dense layers. 

Create a model with one layer of convolution of size 64, one layer of activation, one layer of max pooling with pool size (2,2) and then one flattening layer, one dense layer of unit size 64 with a ReLU activation and one dense output layer. The output layer should have a sigmoid activation.

In [8]:
# Answer below:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5)) #used for regularization to prevent overfitting
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 64)      1792      
_________________________________________________________________
activation (Activation)      (None, 148, 148, 64)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 350464)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                22429760  
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout (Dropout)            (None, 64)                0

Compile the model using RMSprop.

In [9]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])


Fit the model using a fit generator. Use 50 epochs, 25 training steps and 15 validation steps

In [10]:
# Answer below:
#validation_steps=10 or 5
import time

start_time = time.time()
model.fit(
          train_generator, 
          steps_per_epoch=25,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=10)
exec = (time.time() - start_time)

Epoch 1/50
25/25 [==============================] - 757s 31s/step - loss: 13.0422 - accuracy: 0.5248 - val_loss: 0.6957 - val_accuracy: 0.4963
Epoch 2/50
25/25 [==============================] - 19s 746ms/step - loss: 0.6943 - accuracy: 0.4846 - val_loss: 0.6932 - val_accuracy: 0.4988
Epoch 3/50
25/25 [==============================] - 18s 741ms/step - loss: 0.6932 - accuracy: 0.4956 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/50
25/25 [==============================] - 18s 741ms/step - loss: 0.7429 - accuracy: 0.5006 - val_loss: 0.6931 - val_accuracy: 0.5013
Epoch 5/50
25/25 [==============================] - 18s 737ms/step - loss: 0.6964 - accuracy: 0.4982 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 6/50
25/25 [==============================] - 19s 745ms/step - loss: 0.6934 - accuracy: 0.4752 - val_loss: 0.6932 - val_accuracy: 0.4988
Epoch 7/50
25/25 [==============================] - 18s 740ms/step - loss: 0.6930 - accuracy: 0.5189 - val_loss: 0.6932 - val_accuracy: 0.5000

In [13]:
exec_1 = exec
exec_1

1685.880499124527

Create a new model by adding an additional group of convolution, activation and max pooling layers before the flatten layer. Make the convolution layer of unit size 32. Keep everything else the same.

In [11]:
# Answer below:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dropout(rate=0.5)) #
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
activation_3 (Activation)    (None, 148, 148, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 72, 72, 32)        18464     
_________________________________________________________________
activation_4 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 41472)            

Fit and compile the model in the same way you did with the previous model. How did the results improve?

In [15]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

start_time = time.time()
#fitting
model.fit(train_generator, 
          steps_per_epoch=25,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=10)
exec_2 = (time.time() - start_time)

Epoch 1/50
25/25 [==============================] - 19s 749ms/step - loss: 1.2494 - accuracy: 0.4982 - val_loss: 0.6861 - val_accuracy: 0.5038
Epoch 2/50
25/25 [==============================] - 18s 741ms/step - loss: 0.6909 - accuracy: 0.5512 - val_loss: 0.6847 - val_accuracy: 0.6263
Epoch 3/50
25/25 [==============================] - 19s 747ms/step - loss: 0.7102 - accuracy: 0.5984 - val_loss: 0.6600 - val_accuracy: 0.6150
Epoch 4/50
25/25 [==============================] - 19s 745ms/step - loss: 0.6834 - accuracy: 0.6137 - val_loss: 0.6513 - val_accuracy: 0.6375
Epoch 5/50
25/25 [==============================] - 18s 741ms/step - loss: 0.6699 - accuracy: 0.6212 - val_loss: 0.6251 - val_accuracy: 0.6450
Epoch 6/50
25/25 [==============================] - 19s 757ms/step - loss: 0.6996 - accuracy: 0.6295 - val_loss: 0.6379 - val_accuracy: 0.6775
Epoch 7/50
25/25 [==============================] - 19s 751ms/step - loss: 0.6053 - accuracy: 0.7099 - val_loss: 0.6067 - val_accuracy: 0.6762

In [18]:
exec_2

925.4985637664795

Create a new model based on the model above. Add an additional dense layer of size 64 with a ReLU activation after the flatten layer.

In [16]:
# Answer below:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5)) 
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
activation_7 (Activation)    (None, 148, 148, 64)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        18464     
_________________________________________________________________
activation_8 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 41472)            

Fit and compile in the same way as above. Describe the difference in performance and speed.

In [17]:
# Answer below:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

start_time = time.time()
#fitting
model.fit(train_generator, 
          steps_per_epoch=25,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=10)
exec_3 = (time.time() - start_time)

Epoch 1/50
25/25 [==============================] - 20s 758ms/step - loss: 1.5467 - accuracy: 0.5125 - val_loss: 0.6849 - val_accuracy: 0.5263
Epoch 2/50
25/25 [==============================] - 18s 730ms/step - loss: 0.6909 - accuracy: 0.5524 - val_loss: 0.7422 - val_accuracy: 0.5013
Epoch 3/50
25/25 [==============================] - 18s 724ms/step - loss: 0.6944 - accuracy: 0.5786 - val_loss: 0.6444 - val_accuracy: 0.6250
Epoch 4/50
25/25 [==============================] - 18s 733ms/step - loss: 0.6816 - accuracy: 0.5997 - val_loss: 0.6519 - val_accuracy: 0.6313
Epoch 5/50
25/25 [==============================] - 18s 732ms/step - loss: 0.6430 - accuracy: 0.6356 - val_loss: 0.6856 - val_accuracy: 0.5700
Epoch 6/50
25/25 [==============================] - 18s 731ms/step - loss: 0.6182 - accuracy: 0.6678 - val_loss: 0.6226 - val_accuracy: 0.6800
Epoch 7/50
25/25 [==============================] - 18s 708ms/step - loss: 0.5765 - accuracy: 0.7021 - val_loss: 0.6143 - val_accuracy: 0.6425

In [19]:
exec_3

886.284665107727

Fit and compile using the Adam optimizer. Describe the difference in performance between the Adam and RMSprop optimizers.

In [20]:
# Answer below:
model = Sequential()
model.add(Conv2D(64, (3,3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5)) 
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 64)      1792      
_________________________________________________________________
activation_12 (Activation)   (None, 148, 148, 64)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 64)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 32)        18464     
_________________________________________________________________
activation_13 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 41472)            

In [21]:
#using adam for opt
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#fitting
start_time = time.time()
model.fit(train_generator, 
          steps_per_epoch=25,
          epochs=50,
          validation_data=validation_generator,
          validation_steps=10)
exec_4 = (time.time() - start_time)

Epoch 1/50
25/25 [==============================] - 18s 717ms/step - loss: 0.9803 - accuracy: 0.4925 - val_loss: 0.6931 - val_accuracy: 0.5013
Epoch 2/50
25/25 [==============================] - 18s 729ms/step - loss: 0.6930 - accuracy: 0.5087 - val_loss: 0.6922 - val_accuracy: 0.5000
Epoch 3/50
25/25 [==============================] - 17s 694ms/step - loss: 0.6943 - accuracy: 0.5020 - val_loss: 0.6931 - val_accuracy: 0.5163
Epoch 4/50
25/25 [==============================] - 17s 696ms/step - loss: 0.6934 - accuracy: 0.4593 - val_loss: 0.6931 - val_accuracy: 0.5038
Epoch 5/50
25/25 [==============================] - 17s 698ms/step - loss: 0.6931 - accuracy: 0.5030 - val_loss: 0.6931 - val_accuracy: 0.5188
Epoch 6/50
25/25 [==============================] - 17s 694ms/step - loss: 0.6932 - accuracy: 0.5142 - val_loss: 0.6930 - val_accuracy: 0.5088
Epoch 7/50
25/25 [==============================] - 17s 700ms/step - loss: 0.6932 - accuracy: 0.4843 - val_loss: 0.6930 - val_accuracy: 0.5400

In [22]:
exec_4

873.956027507782